In [1]:
# To do for this script:
# Test each helper function
# Make plots of losses to see whether it is converging
# Check shapes of all matrices
# Take a look at the actual data in each matrix
# Take a look at each weight
# Take a look at whether kernel arguments are being sent in correctly
# Test Gaussian kernel
# Time run
# Implement k-fold cross validation
# Implement 'many runs' mode to do 20 runs

In [2]:
# Questions: 
# Do we shuffle the data at each epoch?

In [3]:
# Potential report content
# Talk about effect of dimensionality on overfitting
# Expand the Gaussian kernel into its feature map and speak about the role of c as a regularizer
# Try to answer the question: for what values of C does Gaussian kernel mimic a polynomial kernel? 
# Potentially make a plot for the above

In [4]:
# Import packages
import os
import numpy as np 
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist, pdist, squareform

In [5]:
def load_data(path, name):
    '''
    Takes in a folder path
    and dataset name and loads
    the corresponding dataset
    divided into features X
    and labels Y as numpy arrays.
    '''
    data = np.loadtxt(os.path.join(path, name))
    X, Y = data[:, 1:], data[:, 0]

    return(X, Y)

In [6]:
def shuffle_data(X, Y):
    '''
    Takes in datasets X and Y
    and returns a random permutation
    of these datasets.
    '''
    perm = np.random.permutation(max(Y.shape))
    X = X[perm, :]
    Y = Y[perm]

    return(X, Y)

In [7]:
def show_images(arr, shape=(16, 16)):
    '''
    Takes in a numpy array
    of pixel values and an image shape 
    and displays the associated image.
    '''
    img = arr.reshape(shape)
    imgplot = plt.imshow(img)

In [8]:
def split_data(X, Y, train_percent):
    '''
    Take datasets X and Y and split them 
    into train_percent*100 % training dataset
    and (1 - train_percent)*100 % hold-out dataset.
    '''
    # Calculate no. of training examples based on user specified percentage
    # Here we use 2/3, 1/3 by default as required by the assignment
    n_train = round(train_percent*max(Y.shape))
    
    # Filter the dataframe to get training and testing rows
    X_train = X[:n_train, :]
    Y_train = Y[:n_train]
    
    # Validation set
    X_val = X[n_train:, :]
    Y_val = Y[n_train:]
    
    # Return statement
    return(X_train, X_val, Y_train, Y_val)

In [9]:
def get_polynomial_kernel(X, X_, d):
    '''
    Take in two matrices X and X_ and
    a kernel parameter d and return the
    Gram Matrix K(x, x_) for polynomial 
    kernel with parameter d. Note that 
    this will return polynomials of exactly
    degree d like the assignment says and 
    not a sum of polynomials upto degree d.
    '''
    return(np.power(np.dot(X, X_.T), d))

In [10]:
def get_gaussian_kernel(X, X_, c):
    '''
    Take in two matrices and a kernel
    parameter C and return the Gram matrix
    K(x, x_) for the Gaussian kernel between
    X and X_.
    '''
    # Compute pairwise distances
    K = np.exp(c*(np.einsum('ij,ij->i',X, X)[:,None] + np.einsum('ij,ij->i',X_,X_) - 2*np.dot(X,X_.T)))
    
    # Return statement
    return(K)

In [11]:
def get_k_folds(X, Y, k):
    '''
    Take in two arrays for features
    and corresponding labels respectively
    as well as a user-specified number of
    folds. Return the X and Y arrays divided
    into the k sub-arrays where each sub-array
    is a fold.
    '''
    fold_length = int(np.round(max(X_train.shape)/k))
    X_train_folds = np.split(X_train_folds.copy(), fold_length)
    Y_train_folds = np.split(Y_train_folds.copy(), fold_length)
    
    return(X_train_folds, Y_train_folds)

In [12]:
def get_predictions(alpha, K_examples):
    '''
    Returns raw predictions and class predictions
    given alpha weights and Gram matrix K_examples.
    '''
    # Take the maximum argument in each column
    Y_hat = alpha @ K_examples
    preds = np.argmax(Y_hat, axis = 0)
    
    # Return statement
    return(Y_hat, preds)

In [13]:
def get_accuracy(target, pred):
    '''
    Returns binary accuracy given 
    two arrays containing true values
    and predicted values respectively.
    '''
    return np.sum(target==pred)/max(target.shape)

In [14]:
def get_confusion_matrix(target, pred):
    '''
    Returns a confusion matrix given two
    arrays containing the true values 'target'
    and predicted values 'pred' respectively.
    Interpretation: we put target values in the 
    rows and predicted values in the columns. So 
    we have that for example the element (2, 1) 
    contains all the elements which have true labels
    2 but are classified as 1.
    '''
    # The confusion matrix should be a square matrix with
    # no. of rows and columns equal to the number of unique
    # values in the target: this is what we compute below

    # Compute the no. of unique values in target
    cf_dim = len(np.unique(target))
    
    # Initialize the confusion matrix as zeros
    cf= np.zeros((cf_dim, cf_dim))

    # Now go through each target value
    # Look at the corresponding prediction
    # Update the corresponding cell in the confusion matrix
    for i in range(len(target)):
        cf[int(target[i]) - 1, int(pred[i]) - 1] += 1
    
    # Return statement
    return(cf)

In [15]:
def get_results(history):
    '''
    This function takes in a 
    dictionary containing an
    epoch-wise record of training
    and validation set binary accuracies
    and returns the epoch at which the highest
    binary accuracy was reached on the dev. set 
    along with the associated accuracies on both
    training and dev. set at that epoch.
    '''
    # Store results
    best_epoch = np.array(history["val_accuracies"]).argmax()
    best_training_accuracy = history['accuracies'][best_epoch]
    best_dev_accuracy = history['val_accuracies'][best_epoch]
    
    # Display results
    print(f"best training accuracy: {history['accuracies'][best_epoch]}")
    print(f"best dev accuracy: {history['dev_accuracies'][best_epoch]}")
    print(f"best epoch: {best_epoch}")
    
    return(best_epoch, best_training_accuracy, best_dev_accuracy)

In [16]:
def train_kernel_perceptron(X_train, Y_train, X_val, Y_val, epochs, lr, kernel_type, kernel_args, n_classes):
    '''
    This is the main training loop for
    the kernel perceptron algorithm.
    '''
    # Store a record of training and validation accuracies at each epoch
    history = {
        "train_accuracies": [],
        "val_accuracies": []
    }
    
    # Transform X according to the user specified kernel
    # Can be either polynomial kernel or Gaussian kernel
    # Do this for both training and validation set
    if kernel_type == 'polynomial':
        K_train = get_polynomial_kernel(X_train, X_train, **kernel_args)
        K_val = get_polynomial_kernel(X_train, X_val, **kernel_args)
    
    elif kernel_type == 'gaussian':
        K_train = get_gaussian_kernel(X_train, X_train, **kernel_args)
        K_val = get_gaussian_kernel(X_train, X_val, **kernel_args)
    
    # Initialize alpha weights and store 
    # the number of samples
    alpha = np.zeros((n_classes, K_train.shape[0]))
    n_samples = max(Y_train.shape)
    
    # Run for a fixed user-specified number of epochs
    for epoch in range(epochs):
        
        # Print the epoch number to track progress
        print('This is epoch number: {}'.format(epoch))

        # Do this for each example in the dataset
        for i in range(n_samples):

            # Compute the prediction with the current weights:
            # dim(A) --> (10, 6199), dim(X_train[i, :]) ---> (6199, 1) ====> dim(y_hat) --> 10 X 1
            Y_hat, _ = get_predictions(alpha, K_train[i, :])
            
            # Now first make a matrix Y with dim(Y) ---> (n_classes,) which is only filled with -1
            # Then get the label from the Y_train matrix
            # If this label is 6 then we want to change the 6th index to 1
            Y = np.full(n_classes, -1)
            Y[int(Y_train[i])] = 1
            
            # Compute sign of predictions
            # This is used in the update
            signs = np.ones(Y_hat.shape)
            signs[Y_hat <= 0] = -1
            
            # Check if the prediction is correct against the labels
            # If it is correct we don't need to make any updates: we just move to the next iteration
            # If it is not correct then we update the weights and biases in the direction of the label
            alpha[Y*Y_hat <= 0, i] -= (signs[Y*Y_hat <= 0]) 
            
        # We finally compute predictions and accuracy at the end of each epoch
        # It is a mistake if the class with the highest predicted value does not equal the true label
        # mistakes += int((np.argmax(Y_hat) + 1) != int(Y_train[i]))
        Y_hat_train, preds_train = get_predictions(alpha, K_train)
        train_accuracy = get_accuracy(Y_train, preds_train)
            
        # Now we compute validation predictions
        Y_hat_val, preds_val = get_predictions(alpha, K_val)
        val_accuracy = get_accuracy(Y_val, preds_val)
        
        # We append to the history dictionary as a record
        history['train_accuracies'].append(train_accuracy)
        history['val_accuracies'].append(val_accuracy)
        
        # We print the accuracies at the end of each epoch
        msg = 'The number of {} accuracy on epoch {} is {}...'
        print(msg.format('train', epoch, train_accuracy))
        print(msg.format('validation', epoch, val_accuracy))
    
    # At the end of the entire run we get confusion matrices
    train_cf = get_confusion_matrix(Y_train, preds_train)
    val_cf = get_confusion_matrix(Y_val, preds_val)
        
    # Return statement
    return(train_cf, val_cf, preds_val, preds_train, history)

In [17]:
def run_perceptron_training(epochs, lr, data_path = os.path.join('..', 'data'), name = 'zipcombo.dat', 
                            kernel_type = 'polynomial', d = 5, n_classes=10):
    '''
    Execute the training steps above and generate
    the results that have been specified in the report.
    '''
    # Prepare data for the perceptron
    X, Y = load_data(data_path, name)
    
    # Shuffle the dataset before splitting it
    X, Y = shuffle_data(X, Y)
    
    # Split the data into training and validation set 
    X_train, X_val, Y_train, Y_val = split_data(X, Y, 0.66666)
    
    # Construct kernel arguments dictionary
    if kernel_type == 'polynomial': kernel_args = {'d': d}

    # Call the perceptron training with the given epochs
    results = train_kernel_perceptron(X_train, Y_train, 
                                      X_val, Y_val, epochs, lr, 
                                      kernel_type, kernel_args, n_classes)
    
    # Return statement
    return(results, Y_train, Y_val)

In [ ]:
def run_k_fold_cross_val(epochs, lr, data_path = os.path.join('..', 'data'), 
                         name = 'zipcombo.dat', kernel_type = 'polynomial', 
                         d = 5, n_classes=10, k = 5):
    '''
    Execute the training steps above and generate
    the results that have been specified in the report.
    '''
    # Prepare data for the perceptron
    X, Y = load_data(data_path, name)
    
    # Shuffle the dataset before splitting it
    X, Y = shuffle_data(X, Y)
    
    # Construct kernel arguments dictionary
    if kernel_type == 'polynomial': kernel_args = {'d': d}
    
    # Split the data into training and validation set 
    X_folds, Y_folds = get_k_folds(X, Y, 5)
    
    # Now go through each fold
    # Every fold becomes the hold-out set at least once
    for fold_no, X_fold in enumerate(X_folds):
        
        # Put in the x-values
        X_train = X_folds[:fold_no] + X_folds[fold_no+1:]
        X_val = X_folds[fold_no]
        
        # Put in the Y values
        Y_train = Y_folds[:fold_no] + Y_folds[fold_no+1:]
        Y_val =  Y_val[:fold_no] + Y_val[fold_no+1:]
        
        # Call the perceptron training with the given epochs
        history = train_kernel_perceptron(X_train, Y_train, 
                                          X_val, Y_val, epochs, lr, 
                                          kernel_type, kernel_args, n_classes)
    
    # Return statement
    return(results, Y_train, Y_val)

In [18]:
# Set the random seed for random number generator to ensure reproducibility
np.random.seed(13290138)

# Call training function once
results, Y_train, Y_val = run_kernel_perceptron_training(epochs=10, lr=1)
train_cf, val_cf, preds_val, preds_train, history = results

This is epoch number: 0
The number of train accuracy on epoch 0 is 0.9890304887885143...
The number of validation accuracy on epoch 0 is 0.9593417231364957...
This is epoch number: 1
The number of train accuracy on epoch 1 is 0.9953218261009841...
The number of validation accuracy on epoch 1 is 0.962568570506615...
This is epoch number: 2
The number of train accuracy on epoch 2 is 0.998709469269237...
The number of validation accuracy on epoch 2 is 0.9693449499838658...
This is epoch number: 3
The number of train accuracy on epoch 3 is 0.9995160509759639...
The number of validation accuracy on epoch 3 is 0.9693449499838658...
This is epoch number: 4
The number of train accuracy on epoch 4 is 0.9996773673173093...
The number of validation accuracy on epoch 4 is 0.9706356889319135...
This is epoch number: 5
The number of train accuracy on epoch 5 is 0.9995160509759639...
The number of validation accuracy on epoch 5 is 0.9722491126169732...
This is epoch number: 6
The number of train accu